In [ ]:
!pip install confluent_kafka

In [1]:
from confluent_kafka import Consumer, KafkaException, KafkaError

# Configuración: se debe reemplazar con los valores de cada cuenta
BROKERS  = "sulky-01.srvs.cloudkafka.com:9094,sulky-02.srvs.cloudkafka.com:9094,sulky-03.srvs.cloudkafka.com:9094"
USERNAME = "hqeayvbp"
PASSWORD = "ljw2vm2Kg78lz1MmxfflvKOtYpArayjh"
TOPIC    = "hqeayvbp-click"

In [2]:
# Tópico que será utilizado
topic = TOPIC.split(",")

# Configuración de la comunicación con Kafka
conf = {'bootstrap.servers': BROKERS,
        'group.id': "%s-consumer" % USERNAME,
        'session.timeout.ms': 6000,
        'default.topic.config': {'auto.offset.reset': 'smallest'},
        'security.protocol': 'SASL_SSL',
        'sasl.mechanisms': 'SCRAM-SHA-256',
        'sasl.username': USERNAME,
        'sasl.password': PASSWORD
        }

# Creación de un productor (publica en el tópico)
c = Consumer(**conf)
c.subscribe(topic)

In [ ]:
try:
  while True:
    # Recepción de un nuevo mensaje en el tópico
    msg = c.poll(timeout=1.0)
    
    # Si no hay mensajes nuevos, continuar la revisión
    if msg is None:
        continue
    
    # Si hay algún error o evento
    if msg.error():
        # Si ocurrió un evento
        if msg.error().code() == KafkaError._PARTITION_EOF:
            print(f'%% {msg.topic()} [{msg.partition()}] alcanzó el fin en el offset {msg.offset()}')
        # Si ocurrió un error
        elif msg.error():
            raise KafkaException(msg.error())
    
    # Si hay un nuevo evento (mensaje)
    else:
        # Mostrar el contenido del mensaje
        print(f'%% {msg.topic()} [{msg.partition()}] en offset {msg.offset()} con clave {str(msg.key())}')
        print(msg.value())

except KeyboardInterrupt:
  print('%% Abortado por el usuario')

In [4]:
c.close()